# Ch 5. Monte Carlo Methods

## 환경에 대한 지식은 없다. 실제 경험을 통해 배운다.
- p(s|a), p(s',r|s,a) 등의 환경의 지식이 없다.
- simulation된 에피소드로부터 배운다.
- by averaing sample returns
- 에피소드 하나씩 배워 나가는 incremental learing 방식이다.
- 정책 선순환(GPI: general policy iteration)와 같은 개념이 동일하게 적용되서 최적화(optimality)로 이끈다.


## 5.1 Monte Carlo Prediction ( sample 기반 정책 평가 )

### state value를 averaging sample return으로 배운다.
- 원래 state value는 expected return임
- law of large number 에 의해서 sample이 많으면 average -> expectation으로 된다. 
- episode 내에 등장하는 state에 대한 return을 계속 누적 평균하면 expected return 으로 빠르게 수렴한다.
 - first-visit MC는 episode에서 특정 state가 첫번째 등장했을때만 고려한 것
 - every-visit MC는 episode에서 특정 state가 등장할 때마다 고려한 것

![](./images/ch05/01.png)

### 환경 지식이 있을때라도 MC는 유용하다.
- 환경 지식이 있다하더라도 DP에 필요한 확률들, p(s'r|s,a)와 같은 것을 쉽게 추정하기 어려움
- 반면 episide를 generation하는 것이 쉬운 경우라면, 예를 들어 black-jack, episode기반 MC를 하는 것이 더 잇점이 있다.

### Backup diagram은 episode 그 자체이다.
- 하나의 episode내의 모든 state sequence 자체가 back-up diagram이 된다.
- 즉 각각의 state의 update는 다른 state에 대해 독립적(independent)이다.
 - 나쁜 점은 DP처럼 boost 효과가 없다는 것
 - 좋은 점은 state의 갱신이 다른 state와 무관하므로 계산 시간에 잇점
 - 또다른 좋은 점은 오직 관심있는 영역의 상태 공간에 대한 episode들을 생성하고, 이들을 전체와 무관하게 학습할 수 있다는 점

![](./images/ch05/02.png)

### WildML : blackjack 예제에서의 state-value의 MC 기반 추정 구현
- [Monte Carlo Prediction](https://github.com/dennybritz/reinforcement-learning/tree/master/MC/MC%20Prediction%20Solution.ipynb)

## 5.2 Monte Carlo Estimation of Action Values

### action-value까지 추정해야 정책 개선까지 나아갈 수 있다.
- q(s,a) given policy
- state-action 쌍으로 동일한 원리로 MC를 적용해서 action-value를 추정한다.

### 가장 큰 단점은 never-visit 문제이다.
- state-value 추정과는 다르게 state-action 쌍은 sample episode를 많이 발생시켜도 관측이 전혀 안되는 경우가 허다하다
- 해당 state에서의 모든 action들에 대한 추정을 해줘야 정책 개선을 하는데 용이한데 이렇게 비관측 상황이 많아지는 것은 심각한 문제

### exploraing start로 never-visit 문제에 대처하자
- episode의 시작을 non-zero의 확률로 특정 state-action pair에서 시작시킨다. 
- 이러면 수많은 시뮬레이션을 진행하다보면 모든 action에 대해 관측열이 발생할 것이다.

### non-zero 확률 기반 stochastic 정책이 더 자연스럽다
- explorating start는 단순히 starting poinit만 임의 설정하는 것이므로 실제의 interation with environment와는 동떨어진 조치
- stochastic 정책을 세우고, 모든 action에 대해서 non-zero 확률을 설정하는 것으로 대안




## 5.3 Monte Carlo Control

### MC로 최적 정책으로 근사화하자 
- 정책 선순환(policy evaluation, policy improvment의 순환)의 원리가 동일하게 적용
- Policy evaluation : 주어진 정책을 가지고 MC로 action-value를 추정
- Policy improvment : 정해진 action-value를 가지고 greedy policy를 다음 단계의 정책으로 책정

### 두 개의 가정
- never visit을 해소하기 위해 exploring starts
- 무한의 episode를 발생시켜 action-value를 수렴해서 추정

### 무한한 episode 대신에 적당한 혹은 극단적으로 하나의 episode만 사용
- 적당히 수렴하기 위한 적당한(sufficient) 에피소드를 사용하거나 
- 혹은 극단적으로 단 하나의 episode를 사용해서 정책 선순환 => value iteration과 동일
 - episode-by-episode
 - called by MC-ES

![](./images/ch05/03.png)

### WildML : e-greedy 기반 MC control
- [Monte Carlo Control with Epsilon-Greedy Policies](https://github.com/dennybritz/reinforcement-learning/tree/master/MC/MC%20Control%20with%20Epsilon-Greedy%20Policies%20Solution.ipynb)
 

## 5.4 Monte Carol Control without Exploring Starts

### Exploring Start 대신에 e-greedy 정책을 사용
- policy improvement시에 
 - 1-epsilon의 확률로는 greedy action을 취하지만 
 - epsilon의 확률로는 랜덤하게(동일 확률로) action을 취한다.
- 이를 통해 never-visit 문제에 대처할 수 있다.
- 수학적으로 이렇게 하더라도 항상 정책이 개선됨을 보장한다.
 - 정책이 개선되지 않고 등가가 되는 순간이 최적 정책이 되는 순간
 
![](./images/ch05/04.png)
![](./images/ch05/05.png)


## 5.5 Off-policy prediction via Importance Sampling

###  두마리 토끼를 잡자 : optimal policy 과 exploratory policy
- on-policy 방식에서는 e-soft policy를 가지고 대응
 - 대부분 최적 정책을 지향하나, 가끔씩 탐험 정책을 지향

### 아예 두 개의 별도 정책을 유지하자 : target policy 와 behavior policy
- behavior policy에서 발생시킨 에피소드로 target policy를 개선하자
- target policy는 학습시켜 optimal policy로 발전할 것이다.
- behavior policy는 보다 많은 탐험(exploratory)을 지향하는 정책
- target policy에서 발생시킨 데이터를 가지고 학습하는 것이 아니므로 off-policy 방식이다.

### Off-policy 방식의 특징
- 새로운 개념이 추가되므로 좀 더 어렵다.
- 더 큰 variance가 발생하고, 수렴에 더 느리다.
- 그렇지만 더 강력하고 일반적이다. 
 - on-policy는 behavior policy와 target policy가 같은 special case이다.

### Importance sampling
- 기대값 추정을 다른 분포에서 발생시킨 샘플을 통해 하는 방식

![](./images/ch05/06.png)

### importance sampling ratio in off-policy 방식
- 각 정책에서의 state-action trajectory의 비율 
- 이 비율은 결국 두 개의 정책에만 종속적이고 MDP 상의 state trajectory에 의존하지 않게 된다. 

![](./images/ch05/07.png)


### state-value 추정의 두가지 방식 : ordinary v.s weighted
- ordinary
 - n 개의 episode에서의 importance sampling ratio를 산술 평균
 - single return의 경우에도 ratio가 존재 => unbiased
 - unbiasd estimator but unbonded variance
- weighted
 - single return의 경우에는 ratio가 cancel out => biased
 - bunded/lower variance but biased estimator
 
![](./images/ch05/08.png)
![](./images/ch05/09.png)
![](./images/ch05/10.png)
 




## Remind of incremental learning in Ch 2

![](./images/ch05/11.png)


## Incremental implementation

### incremental MC on episode-by-episode basis
- averaged returns 에 대해서 위의 incremental 방식을 동일하게 적용
- ordinary importance sampling의 경우에는 scaled reward를 동일하게 적용 
- weighted importance sampling의 경우에는 아래의 update 공식을 사용

![](./images/ch05/12.png)

## off-policy, incremental, every-visit, weighted importance sampling 기반 구현

![](./images/ch05/13.png)


## 5.6 Off-policy Monte Carlo Control

### off-policy 기반 control에서는 target policy를 개선한다.
- behavior policy를 기반으로 episode 생성
- 이를 바탕으로 value 추정
- greedy 정책으로 target policy를 개선
 - prob = 1

![](./images/ch05/14.png)

### WildML : Off-policy Monte Carlo Control
- [Monte Carlo Off-Policy Control with Importance Sampling](https://github.com/dennybritz/reinforcement-learning/tree/master/MC/Off-Policy%20MC%20Control%20with%20Weighted%20Importance%20Sampling%20Solution.ipynb)